In [1]:
import tensorflow as tf

create model

### create RNN cell

In [8]:
units = 512
numLayers = 2
dropout = 0.9
max_length_Deco = 12
max_length_enco = 10

In [9]:
def create_rnn_cell():
    enco_deco_cell = tf.contrib.rnn.BasicLSTMCell(units)
    enco_deco_cell = tf.contrib.rnn.DropoutWrapper(
        enco_deco_cell,
        input_keep_prob=1.0,
        output_keep_prob=dropout
    )
enco_deco_cell = tf.contrib.rnn.MultiRNNCell([create_rnn_cell() for _ in range(numLayers)])

In [10]:
encoder_inputs  = [tf.placeholder(tf.int32,   [None, ]) for _ in range(max_length_enco)]

decoder_inputs  = [tf.placeholder(tf.int32,   [None, ], name='inputs') for _ in range(max_length_Deco)]
decoder_targets = [tf.placeholder(tf.int32,   [None, ], name='targets') for _ in range(max_length_Deco)]
decoder_weights = [tf.placeholder(tf.float32, [None, ], name='weights') for _ in range(max_length_Deco)]

### Define the network

In [ ]:
decoderOutputs, states = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
            encoder_inputs,
            decoder_inputs,
            enco_deco_cell,
            textData.getVocabularySize(),
            textData.getVocabularySize(),
            embedding_size=self.args.embeddingSize, 
            output_projection=outputProjection.getWeights() if outputProjection else None,
            feed_previous=bool(self.args.test)
        )